# NRC-09 Darks: Hot and Noisy Pixels

|                                |                                                          |
|:---                            |:---                                                      |
|__CAR Title__                   | NRC-09: Darks                                            |
|__APT Program #__               | 1062                                                     |
|__NGAS #__                      | 102                                                      |
|__CAR Execution Dates(s) (UT):__| TBD                                                      |
|__JIRA Ticket Links:__          | CAR [NRCCOM-13](https://jira.stsci.edu/browse/NRCCOM-13) ||
|                                | CAP [NRCCOM-14](https://jira.stsci.edu/browse/NRCCOM-14) ||
|                                | [JSOCOPS-15](https://jira.stsci.edu/browse/JSOCOPS-15)   ||
|                                | NRCC1C-mm TBD                                            ||
|                                | NRCC1C-nn TBD                                            |
|__Analysis Team/Roles:__        | Leads: Karl Misselt (IDT), Alicia Canipe (IST)           ||
|                                | Jarron Leisenring (Analysis/Scripts)                     ||
|                                | Thomas Beatty (TSO expertise)                            ||
|                                | Bryan Hilbert (Analysis/Scripts)                         ||
|                                | Ben Sunnquist (Analysis/Scripts)                         ||
|                                | Armin Rest (Analysis/Scripts)                            ||

## Table of Contents

* [Objective](#objective)
* [Relevant links and documentation](#links)
* [Environment for analysis](#environment)
* [Imports](#imports)
* [Data for analysis](#data)
* [Superbias investigation](#superbias)

<a id='objective'></a>
## Objective

Use the dark exposures to search for hot pixels and generate hot pixel maps. It might be worth looking for bad pixels in subarrays (i.e., do subarrays have additional bad pixels that are not found in the full-frames?). Compare with ground test data. Generate bad pixel masks.

<a id='links'></a>
## Relevant links and documentation

|                       |                                                                                  |
|:---                   |:---                                                                              |
__JWQL dark monitor__   |                                                                             |
__NRC-09 CAR page__     |[Review Notes NRC-09](https://outerspace.stsci.edu/display/JN/Review+Notes+NRC-09)|
__NRC-09 CAP page__     |[CAP: NIRCam-09](https://outerspace.stsci.edu/display/JN/CAP%3A+NIRCam-09)        |
__Scripts__             |[NIRCam Commissioning Analysis Scripts](https://outerspace.stsci.edu/display/JN/NIRCam+Commissioning+Analysis+Scripts)                            |

<a id='environment'></a>
## Environment for analysis

Follow instructions for downloading the latest version of the pipeline to get the necessary analysis tools. Activate your environment, and then add additional tools. For this analysis, you will also need the bad pixel algorithm in the jwst_reffiles package, which is used to generate bad pixel masks. 

Note that pipeline software is not guaranteed to work on Windows machines, but it *should* work, in theory. 

[Pipeline installation](https://github.com/spacetelescope/jwst#installing-latest-releases)

```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst
```

or for a specific version:
```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst==1.2.3
```

and add additional tools used that are not included in the ```jwst``` package:
```python
pip install ipython jupyter matplotlib pylint pandas
```

and install [jwst_reffiles](https://jwst-reffiles.readthedocs.io/en/latest/official_bad_pixel_mask.html?highlight=hot%20pixel):
```python
pip install git+https://github.com/spacetelescope/jwst_reffiles.git
```

<a id='imports'></a>
## Imports

In [ ]:
import argparse
import requests
import yaml
from glob import glob
from jwst import datamodels
from jwst.datamodels import dqflags
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from jwst_reffiles.bad_pixel_mask.bad_pixel_mask import bad_pixels
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import FuncFormatter, MaxNLocator
%matplotlib inline

<a id='data'></a>
## Data for analysis

Data will be downloaded from MAST using the code written by Mario Gennaro and Armin Rest, and stored in our NIRCam data location: **TBD**

This analysis relies on ```rate```, ```fitopt```, ```jump``` file outputs for the dark exposures.

In [ ]:
# base_dir = '/ifs/jwst/wit/nircam/'
base_dir = '/ifs/jwst/wit/witserv/data7/nrc/'
analysis_dir = './'
ground = 'otis_long_darks'
flight = 'TBD'

In [ ]:
dark_rate_files = sorted(glob(base_dir+'*_rate.fits'))
dark_jump_files = sorted(glob(base_dir+'*_jump.fits'))
dark_fitopt_files = sorted(glob(base_dir+'*_fitopt.fits'))
dark_uncal_files = sorted(glob(base_dir+'*_uncal.fits'))

<a id='hot pixels'></a>
## Hot and noisy pixel investigation

This section uses a collection of dark current exposures in order to search for NOISY, HOT, RC, TELEGRAPH, and LOW_PEDESTAL pixels. Both of these modules from ```jwst_reffiles``` expect input data in at least 2 calibration states: slope images, “fitopt” files, CR-flagged ramps (```rate```, ```fitopt```, ```jump``` files). Detailed [documentation about the software is here](https://jwst-reffiles.readthedocs.io/en/latest/official_bad_pixel_mask.html?highlight=hot%20pixel).

When running bad_pixel_mask.py as a standalone package, the input file lists described above must be provided manually. In the future, when running via ```mkrefs```, raw or partially-calibrated files will be allowed as inputs. ```mkrefs``` will then call the calibration pipeline and produce the needed files.

To use the bad pixel mask generator function as a standalone package, we use the code shown below. Detailed descriptions of the keywords in the call are linked in the documentation. Note that all of the keywords in the call below have defaults defined in the code (and in fact the call below is using all default values), so you do not have to specify any keywords where you wish to use the default.

In [ ]:
bad_pixels(dead_search =True,
           low_qe_and_open_search =True,
           dead_search_type ='sigma_rate',
           smoothing_box_width =15,
           smoothing_type ='Box2D',
           dead_sigma_threshold =5.,
           max_dead_norm_signal =None,
           run_dead_flux_check =False,
           dead_flux_check_files =None,
           flux_check =45000,
           max_low_qe_norm_signal =0.5,
           max_open_adj_norm_signal =1.05,
           manual_flag_file ='default',
           dark_slope_files=dark_rate_files,
           dark_uncal_files=None,
           dark_jump_files=dark_jump_files,
           dark_fitopt_files=dark_fitopt_files,
           dark_stdev_clipping_sigma =5.,
           dark_max_clipping_iters =5,
           dark_noisy_threshold =5,
           max_saturated_fraction =0.5,
           max_jump_limit =10,
           jump_ratio_threshold =5,
           early_cutoff_fraction =0.25,
           pedestal_sigma_threshold =5,
           rc_fraction_threshold =0.8,
           low_pedestal_fraction =0.8,
           high_cr_fraction =0.8,
           flag_values ={'hot': ['HOT'], 'rc': ['RC'], 'low_pedestal': ['OTHER_BAD_PIXEL'], 'high_cr': ["TELEGRAPH"]},
           dark_do_not_use =['hot', 'rc', 'low_pedestal', 'high_cr'],
           plot =False,
           output_file =analysis_dir+'test_bpm.fits',
           author ='jwst_reffiles',
           description ='A bad pix mask',
           pedigree ='GROUND',
           useafter ='2019-04-01 00:00:00',
           history ='',
           quality_check =False)

Now take the outputs from ```mkrefs``` to find the number of pixels flagged for NOISY, HOT, RC, TELEGRAPH, and LOW_PEDESTAL. The mask values recognized by the pipeline are listed [here](https://jwst-pipeline.readthedocs.io/en/stable/jwst/references_general/references_general.html#data-quality-flags). 

In [ ]:
output = datamodels.MaskModel('test_bpm.fits')

### Hot pixels

In [ ]:
hot = np.where(output.dq & dqflags.pixel['HOT'] > 0)

In [ ]:
num_hot_flags = len(hot[0])
print(('Found {} hot flags.'.format(num_hot_flags)))

### Noisy pixels

In [ ]:
noisy = np.where(output.dq & dqflags.pixel['NOISY'] > 0)

In [ ]:
num_noisy_flags = len(noisy[0])
print(('Found {} noisy flags.'.format(num_noisy_flags)))

### RC pixels

In [ ]:
rc = np.where(output.dq & dqflags.pixel['RC'] > 0)

In [ ]:
num_rc_flags = len(rc[0])
print(('Found {} rc flags.'.format(num_rc_flags)))

### Telegraph pixels

In [ ]:
telegraph = np.where(output.dq & dqflags.pixel['TELEGRAPH'] > 0)

In [ ]:
num_telegraph_flags = len(telegraph[0])
print(('Found {} telegraph flags.'.format(num_telegraph_flags)))

### Low pedestal pixels

In [ ]:
low_pedestal = np.where(output.dq & dqflags.pixel['LOW_PEDESTAL'] > 0)

In [ ]:
num_low_pedestal_flags = len(low_pedestal[0])
print(('Found {} low_pedestal flags.'.format(num_low_pedestal_flags)))